<a href="https://colab.research.google.com/github/PremGorecki/MachineLearning/blob/main/01_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd